In [ ]:
#ライブラリのインポート欄
import pandas as pd;import numpy as np;import matplotlib.pyplot as plt;import seaborn as sns
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import warnings;import seaborn as sns #可視化
warnings.filterwarnings('ignore');import zipfile#サンプルがzipなので展開する
zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip').extractall() 
zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip').extractall()
#zipfile.ZipFile('/kaggle/input/ghouls-goblins-and-ghosts-boo/sample_submission.csv.zip').extractall()

%matplotlib inline
from sklearn import datasets
from sklearn.model_selection import train_test_split # クロスバリデーション用（テストとトレ分ける）
from sklearn.model_selection import cross_val_score
from sklearn import metrics       # 精度検証用
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
# ニューラルネットワーク
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
import joblib

from sklearn import svm
from sklearn.linear_model import LogisticRegression


# LightGBM#import lightgbm as lgb
import optuna
import optuna.integration.lightgbm as lgb



from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


In [ ]:
 def N_net(train, test, target):
    # paramate
    hidden_layer_sizes=(100,)
    activation = 'relu'
    solver = 'adam'
    batch_size = 'auto'
    alpha = 0.0001
    random_state = 0
    max_iter = 10000
    early_stopping = True

    # 学習
    clf = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        batch_size=batch_size,
        alpha=alpha,
        random_state=random_state,
        max_iter=max_iter,
    #     early_stopping = early_stopping
        )

    clf.fit(train, target)
    SAVE_TRAINED_DATA_PATH = 'train1.learn'
    # 学習結果を出力
    joblib.dump(clf, SAVE_TRAINED_DATA_PATH)
    # 学習済ファイルのロード
    clf1 = joblib.load(SAVE_TRAINED_DATA_PATH)
    # 学習結果の検証
    #predict_y1 = clf1.predict_proba(test)それぞれの回答確率を出す?
    predict_no = clf1.predict(test)
    predict = np.round(np.round(predict_no, decimals=1)) # 最尤と判断したクラスの値にする
    return  predict

In [ ]:
def N_netlog(train, test, target):
    # paramate
    hidden_layer_sizes=(100,)
    activation = 'relu'
    solver = 'adam'
    batch_size = 'auto'
    alpha = 0.0001
    random_state = 0
    max_iter = 10000
    early_stopping = True
    # 学習
    clf = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        batch_size=batch_size,
        alpha=alpha,
        random_state=random_state,
        max_iter=max_iter,
    #     early_stopping = early_stopping
        )
    clf.fit(train, target)
    SAVE_TRAINED_DATA_PATH = 'train1.learn'
    # 学習結果を出力
    joblib.dump(clf, SAVE_TRAINED_DATA_PATH)
    # 学習済ファイルのロード
    clf1 = joblib.load(SAVE_TRAINED_DATA_PATH)
    # 学習結果の検証
    #predict_y1 = clf1.predict_proba(test)それぞれの回答確率を出す?
    predict = clf1.predict(test)
    return  predict


In [ ]:
def lightgbm(train, test, target):# データ用意

    X_train, X_test, Y_train, Y_test = train_test_split(train, target, random_state=0) # random_stateはseed値。
    # LightGBMのパラメータ設定
    params = {
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': 'multi_error',#クラスは0,1,2,...と与えられる(数字は関係ない)#評価指標：正答率
        'num_class': 3,
        #'num_iterations': 1000,#1000回学習
        'verbose': -1 #学習情報を非表示
    }
    #'metric': 'multi_logress'かえた
    # LightGBMを利用するのに必要なフォーマットに変換
    lgb_train = lgb.Dataset(X_train, Y_train)
    lgb_eval = lgb.Dataset(X_test, Y_test, reference=lgb_train)
    best_params, history = {}, []

    # LightGBM学習
    gbm = lgb.train(params,
                    lgb_train,
                    valid_sets=[lgb_train, lgb_eval],
                    verbose_eval=100,
                    early_stopping_rounds=100
                   )

    best_params = gbm.params
    print("Best params:", best_params)
    params = best_params
    #ここから推理します

    predict_no = gbm.predict(test, num_iteration=gbm.best_iteration)# LightGBM推論
    predict = np.argmax(predict_no, axis=1)  # 最尤と判断したクラスの値にする

    return predict


In [ ]:
def SVM(train, test, target):
    clf_result=svm.SVC(kernel='rbf', gamma=1/2 , C=1.0,class_weight='balanced', random_state=0) #loss='squared_hinge' #loss="hinge", loss="log"
    
   # clf_result=svm.LinearSVC(loss='hinge', C=7.0,class_weight='balanced', random_state=0)#loss='squared_hinge' #loss="hinge", loss="log"
    clf_result.fit(train, target)
    predict= np.array(clf_result.predict(test))
    return predict

In [ ]:
def LogRes(ghost,ghoul,goblin, test, target,HowDo):#おかしい。うまく行っていない
    #HowDoはどの分析方法にするか
    vsnp= np.empty((529,6),dtype="float64")
    submission=np.empty((529),dtype="int")
    ghost0=np.zeros(len(ghost));ghost1=np.ones(len(ghost))
    ghoul0=np.zeros(len(ghoul));ghoul1=np.ones(len(ghoul))
    goblin0=np.zeros(len(goblin));goblin1=np.ones(len(goblin))#target作成前段階
    vs1 = ghost.append(ghoul, ignore_index=True);vs1t = np.append(ghost0,ghoul1)
    vs2 = ghoul.append(goblin, ignore_index=True);vs2t = np.append(ghoul0,goblin1)
    vs3 = goblin.append(ghost, ignore_index=True);vs3t = np.append(goblin0,ghost1)#train作成;#target作成
    if HowDo == "log":
        model1 = LogisticRegression();model1.fit(vs1,vs1t);vsnp[:,0:2]=model1.predict_proba(test)
        model2 = LogisticRegression();model2.fit(vs2,vs2t);vsnp[:,2:4]=model2.predict_proba(test)
        model3 = LogisticRegression();model3.fit(vs3,vs3t);vsnp[:,4:6]=model3.predict_proba(test)
    if HowDo == "net":
        vsnp[:,0]=1-N_netlog(vs1, test, vs1t);vsnp[:,1]=1-vsnp[:,0]
        vsnp[:,2]=1-N_netlog(vs2, test, vs2t);vsnp[:,3]=1-vsnp[:,2]
        vsnp[:,4]=1-N_netlog(vs3, test, vs3t);vsnp[:,5]=1-vsnp[:,4]
        
        
    
    for n in range(len(vsnp)):
        if np.argmax(vsnp[n,:])==0:
            submission[n]=0
        if np.argmax(vsnp[n,:])==1:
            submission[n]=2
        if np.argmax(vsnp[n,:])==2:
            submission[n]=2
        if np.argmax(vsnp[n,:])==3:
            submission[n]=1
        if np.argmax(vsnp[n,:])==4:
            submission[n]=1
        if np.argmax(vsnp[n,:])==5:
            submission[n]=0
    print(vsnp)
    return submission

In [ ]:
def LogRes2(ghost,ghoul,goblin, test, target,HowDo):
    vsnp=np.empty((529,3),dtype="float64")
    submission=np.empty((529),dtype="int")
    ghost0=np.zeros(len(ghost));ghost1=np.ones(len(ghost))
    ghoul0=np.zeros(len(ghoul));ghoul1=np.ones(len(ghoul))
    goblin0=np.zeros(len(goblin));goblin1=np.ones(len(goblin))#target作成前段階
    vs1 = ghost.append(ghoul, ignore_index=True)
    vs1 = vs1.append(goblin, ignore_index=True)
    vs1t = np.append(ghost0,ghoul1)
    vs1t = np.append(vs1t,goblin1)
    vs2 = ghoul.append(goblin, ignore_index=True)
    vs2 = vs2.append(ghost, ignore_index=True)
    vs2t = np.append(ghoul0,goblin1)
    vs2t = np.append(vs2t,ghost1)
    vs3 = goblin.append(ghost, ignore_index=True)#train作成
    vs3 = vs3.append(ghoul, ignore_index=True)#train作成
    vs3t = np.append(goblin0,ghost1)#target作成
    vs3t = np.append(vs3t,ghoul1)
    if HowDo == "log":
        model1 = LogisticRegression();model1.fit(vs1,vs1t);vsnp[:,0]=model1.predict_proba(test)[:,0]
        model2 = LogisticRegression();model2.fit(vs2,vs2t);vsnp[:,1]=model2.predict_proba(test)[:,0]
        model3 = LogisticRegression();model3.fit(vs3,vs3t);vsnp[:,2]=model3.predict_proba(test)[:,0]

    if HowDo == "net":
        vsnp[:,0]=1-N_netlog(vs1, test, vs1t)
        vsnp[:,1]=1-N_netlog(vs2, test, vs2t)
        vsnp[:,2]=1-N_netlog(vs3, test, vs3t)
    
    
    for n in range(len(vsnp)):
        if np.argmax(vsnp[n,:])==0:
            submission[n]=0
        if np.argmax(vsnp[n,:])==1:
            submission[n]=2
        if np.argmax(vsnp[n,:])==2:
            submission[n]=1
    print(vsnp)
    return submission

In [ ]:
def syuunou(vote, ID):
    pred:str=[]
    for n in range(len(ID)):
        if np.argmax(vote[:,n])==0:
            pred.append('Ghost')
        if np.argmax(vote[:,n])==2:
            pred.append('Ghoul')
        if np.argmax(vote[:,n])==1:
            pred.append('Goblin')
    s_c= pd.DataFrame({"id": ID, "type": pred})
    return s_c

In [ ]:
def tohyo(predict, vote):
    vote[0] += (predict==0);vote[1] += (predict==1);vote[2] += (predict==2)

In [ ]:
def main_n():
   # CSVを読み込む
    train = pd.read_csv('./train.csv')
    test = pd.read_csv('./test.csv')
    
    #type_pd = train["type"]
    type_array = pd.get_dummies(train['type']);  del train['type']#typeをトレインから分離
    COLOR = pd.get_dummies(train['color']);  del train['color']  ;del train['id']#colorをトレインから分離；idをトレインから分離
    COLOR2 = pd.get_dummies(test['color']);  del test['color'];  ID = test["id"];  del test['id'] #testも同じようにする
    vote = np.zeros((3,529),dtype = 'int')
    target = pd.DataFrame(type_array['Ghost']* 0 + type_array['Ghoul'] * 2 + type_array['Goblin'] * 1)#targetを作成する
    #怪物のデータが別々にいるプログラム用
    ghost=train[type_array['Ghost']==1]
    ghoul=train[type_array['Ghoul']==1]
    goblin=train[type_array['Goblin']==1]
    
    
    tohyo(N_net(train, test, target), vote)#ニューラルネットの予想
    tohyo(lightgbm(train, test, target), vote)#libhtGBM+oputunaの予想
    tohyo(SVM(train, test, target), vote)#libhtGBM+oputunaの予想
    #tohyo(LogRes(ghost,ghoul,goblin, test, target,"log"), vote)#libhtGBM+oputunaの予想(タイプ別なので注意)
    tohyo(LogRes2(ghost,ghoul,goblin, test, target,"log"), vote)#libhtGBM+oputunaの予想(タイプ別なので注意)
    tohyo(LogRes2(ghost,ghoul,goblin, test, target,"net"), vote)#libhtGBM+oputunaの予想(タイプ別なので注意)
    #ここからは色つきでもう一度同じシミュレーション
    train = train.join(COLOR)
    test = test.join(COLOR2)
    #怪物のデータが別々にいるプログラム用
    ghost=train[type_array['Ghost']==1]
    ghoul=train[type_array['Ghoul']==1]
    goblin=train[type_array['Goblin']==1]
    tohyo(LogRes2(ghost,ghoul,goblin, test, target,"net"), vote)#libhtGBM+oputunaの予想(タイプ別なので注意)
    tohyo(N_net(train, test, target), vote)#ニューラルネットの予想
    #tohyo(lightgbm(train, test, target), vote)#libhtGBM+oputunaの予想
    #tohyo(SVM(train, test, target), vote)#libhtGBM+oputunaの予想
    #tohyo(LogRes(ghost,ghoul,goblin, test, target,"log"), vote)#libhtGBM+oputunaの予想(タイプ別なので注意)
    submission = syuunou(vote, ID)
    return submission

In [ ]:
#ここでメインを一つずつ実行
submission=main_n()
rows=submission
##ここから推理します
# Kaggle提出用csvファイルの作成
#submission = pd.DataFrame({"id": ID, "type": y_preda})
submission = rows
submission.to_csv("submission6.csv", index=False)

ここから使っていないコード

In [ ]:
'''
def color_select():
    global train
    global test
    global target
    global train_k
    global test_k
    global target_k
    global N_net
    global syuunou
    global ID_black
    global y_pred_AA
    train_k = train
    test_k = test
    target_k = target
    ID_black = ID
    N_net()
    syuunou()
    
#各色で選択する
#GOB()
color_select()
sc = s_c
#並べ替え　初期設定
rows = sc.sort_values('id')
'''

In [ ]:


    

'''
def GOB():
    global y_pred_c
    global train
    global test
    # paramate
    hidden_layer_sizes=(100,)
    activation = 'relu'
    solver = 'adam'
    batch_size = 'auto'
    alpha = 0.0001
    random_state = 0
    max_iter = 10000
    early_stopping = True
    train_X = train_c# 説明変数
    train_y1 = target_GOB# 目的変数
    # 学習
    clf = MLPRegressor(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        batch_size=batch_size,
        alpha=alpha,
        random_state=random_state,
        max_iter=max_iter,
    #     early_stopping = early_stopping
        )

    clf.fit(train_X, train_y1)
    SAVE_TRAINED_DATA_PATH = 'train1.learn'
    joblib.dump(clf, SAVE_TRAINED_DATA_PATH)# 学習結果を出力
    clf1 = joblib.load(SAVE_TRAINED_DATA_PATH)# 学習済ファイルのロード
    
    test_X1 = test_c# 検証用の説明変数
    
    # 学習結果の検証
    #predict_y1 = clf1.predict_proba(train_X)
    predict_Y1 = clf1.predict(train_X)
    print(predict_Y1)
    
    y_pred_c = pd.DataFrame(predict_Y1)
    train = train.join(y_pred_c)
    
    #predict_y1 = clf1.predict_proba(test_X1)
    predict_Y1 = clf1.predict(test_X1)
    print(predict_Y1)
    y_pred_c = pd.DataFrame(predict_Y1)
    test = test.join(y_pred_c)
#できた結果をつなぐ
dataset = pd.concat([train, test], ignore_index = True)
'''

In [ ]:
'''
type_array = pd.get_dummies(train['type']);  del train['type']
COLOR = pd.get_dummies(train['color']);  del train['color']  ;del train['id']
#ダミー化した結果をクラスとして格納
target = pd.DataFrame(type_array['Ghost']* 0 + type_array['Ghoul'] * 2 + type_array['Goblin'] * 1)
target_GOB = pd.DataFrame(type_array['Ghost']* 0 + type_array['Ghoul'] * 0 + type_array['Goblin'] * 1)
target = target.rename(columns={'0': 'target'})
#予想を表から除いてダミー化(テストデータも)
COLOR2 = pd.get_dummies(test['color'])
del test['color'];  ID = test["id"];  del test['id']

#ゴブリンだけ色を入れて予想(色付き予想のために使うもの)
train_c = train.join(COLOR)
test_c = test.join(COLOR2)
#target_c = target.join(COLOR)
'''